In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv3D, Flatten,MaxPooling3D,AveragePooling3D, concatenate,Input ,SpatialDropout3D,Dropout
import keras
import math
from math import e
from sklearn.model_selection import train_test_split
from keras.models import Model
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
from sklearn.model_selection import KFold

In [4]:
fogData = xr.open_dataset('../../../Data/FogData/CombinedFogData_25Stations.nc')
StackFog=fogData.fogdata.stack(a=('years','months','days'))
StackFog
dd =[];
for i in range(StackFog.years.values.shape[0]):
    dd=dd+[str(StackFog.years[i].values)+'-'+str(StackFog.months[i].values)+"-"+str(StackFog.days[i].values)]
fg = xr.Dataset({'fogdata': (('time','stations'), StackFog.values.T)}, coords={'time': pd.to_datetime(dd),'stations': fogData.stations})
yAll=fg.fogdata.sum(dim='stations').sel(time=slice('1980-1-1','2018-12-31'))
yAll=yAll/25.0

In [5]:
t2mData = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_t2m_daily_NovDecJan.nc')

In [8]:
t2mData.t2m.coords

Coordinates:
  * time       (time) datetime64[ns] 1979-01-01T09:00:00 ... 2019-12-31T09:00:00
  * longitude  (longitude) float32 0.0 2.0 4.0 6.0 ... 352.0 354.0 356.0 358.0
  * latitude   (latitude) float32 90.0 88.0 86.0 84.0 82.0 ... 6.0 4.0 2.0 0.0

In [10]:
#Surface data
t2mData = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_t2m_daily_NovDecJan.nc')
d2mData = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_d2m_daily_NovDecJan.nc')
u10Data = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_u10_daily_NovDecJan.nc')
v10Data = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_v10_daily_NovDecJan.nc')
# 850 Data
tuv850Data = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_850_uvt_daily_NovDecJan.nc')
z850Data = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_850_z_daily_NovDecJan.nc')
#700 Data
w700Data = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_700_w_daily_NovDecJan.nc')
# Read Orography
OroData = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5IGP_Orography.nc')
z500Data = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_500_z_daily_NovDecJan.nc')

In [13]:
# Calculate wind speed 
ws = ((v10Data.v10.values**2)+(u10Data.u10.values**2))**0.5
ws_ds = xr.Dataset({'ws': (('time','latitude','longitude'), ws)}, coords={'time': t2mData.time,'latitude': t2mData.latitude,'longitude': t2mData.longitude})


In [18]:
testData=ws_ds.ws.sel(latitude=slice(35,0),longitude=slice(50,100))

In [19]:
testData.mean()

<xarray.DataArray 'ws' (time: 3772, latitude: 18, longitude: 26)>
array([[[2.0672154 , 0.24223132, 1.0973994 , ..., 0.40268904,
         1.7176    , 2.6787457 ],
        [0.50542855, 1.3449271 , 0.93851525, ..., 0.84729326,
         1.3655972 , 1.0970205 ],
        [3.5402017 , 1.2489607 , 2.5659797 , ..., 0.86544204,
         1.3846198 , 2.3707466 ],
        ...,
        [7.212421  , 5.8544164 , 5.7561297 , ..., 1.0576451 ,
         0.5099486 , 2.009115  ],
        [5.7871723 , 4.6096334 , 4.1348743 , ..., 1.98117   ,
         1.1448476 , 0.66691387],
        [4.542605  , 3.3244789 , 3.4845757 , ..., 0.30131274,
         0.20552962, 0.26754105]],

       [[2.9059405 , 0.68705755, 2.6016526 , ..., 1.5047784 ,
         3.9237306 , 2.2744691 ],
        [1.0801065 , 2.0346072 , 1.3939866 , ..., 0.9962449 ,
         1.4587107 , 1.2770814 ],
        [2.5646143 , 1.7344061 , 1.8729552 , ..., 1.5264195 ,
         1.3122575 , 2.485501  ],
...
        [6.9898367 , 6.888002  , 6.8406343 , ..., 0.13041498,
         0.06403647, 0.8825194 ],
        [7.12302   , 6.181293  , 5.340234  , ..., 2.699342  ,
         1.929749  , 0.57461375],
        [6.6791887 , 5.8911996 , 5.333205  , ..., 2.1625247 ,
         1.7580813 , 0.43755567]],

       [[2.3199635 , 0.99274236, 0.8094134 , ..., 3.1508012 ,
         4.775285  , 2.0039837 ],
        [0.44753766, 3.1310937 , 1.6028031 , ..., 1.5103235 ,
         0.7833432 , 1.6829952 ],
        [2.7853992 , 2.2931845 , 2.792979  , ..., 0.8865809 ,
         2.5260403 , 3.7966506 ],
        ...,
        [8.029464  , 7.59603   , 6.9180717 , ..., 0.9405948 ,
         0.383588  , 1.7600092 ],
        [7.6656203 , 6.7238593 , 5.837408  , ..., 0.7279783 ,
         1.3505262 , 0.32000247],
        [7.0501504 , 5.8894153 , 4.969997  , ..., 1.4360739 ,
         1.9277569 , 0.9606149 ]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 1979-01-01T09:00:00 ... 2019-12-31T09:00:00
  * latitude   (latitude) float32 34.0 32.0 30.0 28.0 26.0 ... 6.0 4.0 2.0 0.0
  * longitude  (longitude) float32 50.0 52.0 54.0 56.0 ... 94.0 96.0 98.0 100.0